# Functional Test 3.1.2 - Smart NICs

This Jupyter notebook will allow you to create VMs on different sites and worker nodes consistent with requirements for test 3.1.2 for testing Smart NIC attachment.

## Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../fablib_api/configure_environment/configure_environment.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

**This only needs to be done once.**

## Step 2: Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

## Step 3: Check your existing slices

Since testing can get confusing, check what slices you actually have. It may print nothing if you have no active slices.

In [ ]:
try:
    for slice in fablib.get_slices():
        print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Test Slice

The following creates a node with Smart NICs. Orchestrator will auto-select appropriate workers for each test based on the requested NIC types. If you are unsure, the generated ads for each site ([in JSON format](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON)) can help.

A node with specified NIC components (2 each as there are typically 2 NICs per worker) is created. 

Available NIC types are:
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports) 
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports) 

**You should try different nodes and different NIC types**

**The code to create the slice will auto-refresh until the slice is created or it fails**

**You should change the NIC type depending on which worker you are testing.**

In [ ]:
from datetime import datetime
from dateutil import tz

name='Node1'
nic1_name='SmartNIC1'
nic2_name='SmartNIC2'

nic_type='NIC_ConnectX_5'

site='TACC'

cores=10
ram=20
disk=50
slice_name=f"Slice Test 3.1.2-SmartNIC {site} on {datetime.now()}"

In [ ]:
try:
    #Create Slice
    print(f'Creating slice {slice_name}')
    slice = fablib.new_slice(name=slice_name)

    # Node1
    node1 = slice.add_node(name=name, site=site)

    iface1 = node1.add_component(model=nic_type, name=nic1_name).get_interfaces()[0]
    iface2 = node1.add_component(model=nic_type, name=nic2_name).get_interfaces()[0]
 
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")

## Step 5: Observe the Slice's Attributes

### Print the slice 

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Print the Interfaces

You should see 4 interfaces (2 cards, 2 ports).

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

### Smart NIC PCI Devices

Run the command <code>lspci</code> to see your Smart NIC PCI device(s).

View node1's Shared NIC - you should see 4 of either
- `Ethernet controller: Mellanox Technologies MT28908 Family [ConnectX-6]` 
- `Ethernet controller: Mellanox Technologies MT27800 Family [ConnectX-5]`

In [ ]:
command = "sudo dnf install -q -y pciutils && lspci"
try:
    stdout, stderr = node.execute(command)
    print(f"stdout: {stdout}")
except Exception as e:
    print(f"Exception: {e}")

## Step 6: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")